In [ ]:
from IPython.display import clear_output

! pip install trulens_eval==0.20.3 chromadb==0.4.18 openai==1.3.7
clear_output()

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')

# Custom RAG
##### Replace this section with Nuoc's RAG built by Enosta

In [ ]:
city_info = """
Ho Chi Minh City (HCMC, Vietnamese: Thành phố Hồ Chí Minh), formerly known as Saigon (Vietnamese: Sài Gòn), is the most populous city in Vietnam, with
a population of around 9.3 million in 2023. The city's geography is defined by rivers and canals, the largest of which is the eponymously-named Saigon
River. As a municipality, Ho Chi Minh City consists of 16 urban districts, 6 rural districts, and 1 municipal city (Thủ Đức). As the largest financial
centre in Vietnam, Ho Chi Minh City has the highest gross regional domestic product out of all Vietnam provinces and municipalities, contributing
around a quarter of the country's total GDP. Ho Chi Minh City metropolitan is ASEAN 6th largest economy, also the biggest outside ASEAN country capital.
"""

### Vector Store

In [ ]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=city_info
    )

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Cities",
                                                      embedding_function=embedding_function)

In [ ]:
vector_store.add("city_info", documents=city_info)

##### *Need to add TruLens custom instrumentation to Nuoc's RAG (i.e. next cell is necessary)

In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

##### *Nuoc RAG needs to have EXACTLY have the `retrieve`, `generate_completion`, and `query` methods, as well as needs to incorporate the `instrument` decorator

In [ ]:
class CustomRAG:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"We have provided context information below. \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please answer the question: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = CustomRAG()

# TruLens Evaluation

##### Set up feedback functions

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

##### Construct TruLens app

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

##### Run TruLens app

In [ ]:
with tru_rag as recording:
    rag.query("What was Saigon's population in 2023")

In [ ]:
tru.get_leaderboard(app_ids=["RAG v1"])

In [ ]:
tru.run_dashboard()